In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Bone_Density"
cohort = "GSE56815"

# Input paths
in_trait_dir = "../../input/GEO/Bone_Density"
in_cohort_dir = "../../input/GEO/Bone_Density/GSE56815"

# Output paths
out_data_file = "../../output/preprocess/Bone_Density/GSE56815.csv"
out_gene_data_file = "../../output/preprocess/Bone_Density/gene_data/GSE56815.csv"
out_clinical_data_file = "../../output/preprocess/Bone_Density/clinical_data/GSE56815.csv"
json_path = "../../output/preprocess/Bone_Density/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from Affymetrix arrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (bone mineral density): available at index 1
trait_row = 1
# For age: not available in the sample characteristics
age_row = None
# For gender: all subjects are female (constant), so we consider it not available
gender_row = None

# 2.2 Data Type Conversion Functions
def convert_trait(value):
    """Convert bone mineral density values to binary (0 for low BMD, 1 for high BMD)"""
    if value is None:
        return None
    
    # Extract the value part after the colon if present
    if isinstance(value, str) and ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if isinstance(value, str):
        value = value.lower()
        if 'high' in value:
            return 1
        elif 'low' in value:
            return 0
    
    return None

def convert_age(value):
    """Convert age values to continuous numbers"""
    # Not used as age data is not available
    return None

def convert_gender(value):
    """Convert gender values to binary (0 for female, 1 for male)"""
    # Not used as gender data is not available
    return None

# 3. Save Metadata - Initial Filtering
# Since trait_row is not None, trait data is available
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait data is available, extract clinical features
if trait_row is not None:
    # Extract clinical features using the library function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(selected_clinical_df)
    print("Preview of extracted clinical data:")
    print(preview)
    
    # Save the extracted clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# Analyze the gene identifiers in the index
# Looking at the identifiers like '1007_s_at', '1053_at', etc.
# These appear to be probe IDs from Affymetrix microarrays, not standard human gene symbols

# Probe IDs in the format of '1007_s_at' are typical for Affymetrix platforms
# They need to be mapped to human gene symbols for meaningful analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Identify columns in gene annotation data that contain probe IDs and gene symbols
# From the preview, we can see that 'ID' contains probe IDs like '1007_s_at'
# and 'Gene Symbol' contains the gene symbols like 'DDR1 /// MIR4640'
probe_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get gene mapping dataframe using the get_gene_mapping function from the library
mapping_df = get_gene_mapping(gene_annotation, prob_col=probe_col, gene_col=gene_col)

# 3. Convert probe-level measurements to gene-level expression by applying the mapping
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the resulting gene expression dataframe
print(f"Number of genes after mapping: {len(gene_data)}")
print("First 10 gene symbols:")
print(gene_data.index[:10])


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_df = pd.read_csv(out_clinical_data_file)
print("Clinical data structure:")
print(clinical_df.head())

# Get trait column name (actual name in the DataFrame)
trait_column = clinical_df.columns[0]  # First column should be Bone_Density

# Create a DataFrame with correct structure for geo_link_clinical_genetic_data
# The function expects clinical_df to have features as rows and samples as columns
clinical_df_for_linking = pd.DataFrame(clinical_df[trait_column].values[None, :], 
                                      index=[trait], 
                                      columns=clinical_df.index)
print("Clinical data prepared for linking:")
print(clinical_df_for_linking.head())

# Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_df_for_linking, normalized_gene_data)

# For analysis functions, we need samples as rows and features as columns
linked_data_for_analysis = linked_data.T

# 3. Handle missing values in the linked data
linked_data_for_analysis = handle_missing_values(linked_data_for_analysis, trait)

# 4. Determine whether the trait and demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_for_analysis, trait)

# 5. Conduct final quality validation and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from blood monocytes in pre- and postmenopausal females with low or high bone mineral density."
)

# 6. If the linked data is usable, save it as CSV
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data and prepare for linking
# Read the clinical data without setting index_col to avoid issues
clinical_df = pd.read_csv(out_clinical_data_file)

# Get the structure of the clinical data to understand column names
print("Clinical data columns:", clinical_df.columns.tolist())
print("Clinical data shape:", clinical_df.shape)

# Transform clinical data to have the correct format for linking
# The clinical data appears to have samples as columns with the trait value in each row 0
sample_names = clinical_df.columns.tolist()
trait_values = clinical_df.iloc[0].tolist()
clinical_df_transformed = pd.DataFrame({trait: trait_values}, index=sample_names)

# Link clinical and genetic data
linked_data = pd.merge(clinical_df_transformed, normalized_gene_data.T, 
                      left_index=True, right_index=True)

# Check the structure of the linked data
print("Linked data shape:", linked_data.shape)
print("Linked data columns include trait column?", trait in linked_data.columns)

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)

# 4. Determine whether the trait and some demographic features are severely biased
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct final quality validation and save relevant information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from blood monocytes in pre- and postmenopausal females with low or high bone mineral density."
)

# 6. If the linked data is usable, save it as CSV
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved")